# Caso 5: localización de depósitos con múltiples objetivos

---


## Instrucciones generales

El primer paso antes de resolver este laboratorio es leer y entender el **enunciado del caso**.

Este laboratorio tiene las siguientes secciones: 
* **Formulación**: en este caso particular, definimos dos funciones objetivo $z_1$ y $z_2$
* **Importación de librerías**
* **Creación de parámetros**
* **Modelado**: en esta práctica, haremos tres (3) implementaciones del mismo problema:
    * **Minimización de costos**
    * **Maximización de la satisfacción**
    * **Maximización de la satisfacción con restricción de costos**
* **Reporte de Resultados**

Este tipo de actividades se evaluará sobre un total de 100 puntos. Las celdas calificables se distinguen por tener la instrucción `# your code here`. Antes de estas celdas encontrarás instrucciones y consejos para resolver las preguntas, también el puntaje que le corresponde.

¡Éxitos!

## Formulación
---

Te presentamos la formulación del caso de la semana de forma resumida. Te recomendamos revisar la formulación una vez hayas leído el enunciado del caso. Es bueno que te familiarices con los elementos de la formulación antes de iniciar la implementación.

### Conjuntos y Parámetros
>#### **Conjuntos**
>* $I:$ Depósitos
>* $J:$ Centros de acopio consolidados (CACs)

>#### **Parámetros**
>* $k_i:$ Capacidad del depósito $i \in I$ (miles de toneladas por año)
>* $f_i:$ Costo de operación del depósito $i \in I$ (millones de pesos por año)
>* $d_j:$ Producción proyectada del CAC $j\in J$ (miles de tonaledas por año)
>* $q:$ Costo anualizado (por cada mil toneladas por kilómetro) de transportar café
>* $r:$ Distancia máxima (en kilómetros) entre el CAC y el depósito asignado para estar "bien" atendido
>* $h_{ij}:$ Distancia (en kilómetros) entre el CAC $j \in J$ y el depósito $i \in I$
>* $c_{ij}:$ Costo anualizado de atender el CAC $j\in J$ con el depósito $i\in I$ (Se calcula como: $c_{ij} = q\cdot d_j \cdot h_{ij}$)

### Variables de Decisión
>* $x_{ij}=\begin{cases}1, & \text{si el CAC } j \in J \text{ es atendido por el depósito } i \in I \\0, & \text{de lo contrario} \end{cases}$
>* $y_{i}=\begin{cases} 1, & \text{si se decide operar el depósito } i \in I  \\ 0, & \text{de lo contrario} \end{cases}$
    
### Restricciones
>1. Cada CAC debe ser atendido por un único depósito 
>>`# Para desarrollo del estudiante`
>2. No se debe superar la capacidad de los depósitos y sólo se puede atender CACs desde un depósito si se decide operar el mismo.
>>$\sum_{j \in J}d_{j}x_{ij} \leq k_{i}y_{i}, \; \forall i \in I$

> **Naturaleza de variables**
>>$x_{ij} \in \{0,1\} , \;\forall i \in I, j \in J$
>>
>>$y_{i} \in \{0,1\} , \;\forall i \in I$

### Función Objetivo
>* Minimizar los costos totales de operación y transporte
>>`# Para desarrollo del estudiante`
>* Maximizar satisfacción de los CACs
>>$\max z_2 = \sum_{j \in J} d_{j} \sum_{\{i \in I | h_{ij} \leq r\}} x_{ij}$

## Importación de librerías
---
En esta práctica usaremos:
* El paquete `pandas` es muy útil para el análisis de datos en general. Le asignamos el alias de `pd`.
* El paquete `pulp` permite crear modelos de optimización, crear variables, añadir restricciones y muchos más. Le asignamos el alias de `lp`.
* La función `distance` del módulo `geopy.distance` nos permite hallar fácilmente la distancia geodéisca en kilómetros entre dos pares de coordenadas de longitud y latitud.


In [1]:
import pandas as pd
import pulp as lp 
from geopy.distance import distance

## Creación de Parámetros
---

### Lectura del archivo de soporte

Los datos que necesitamos para esta práctica se encuentran disponibles en el archivo `Soporte Caso 5.xlsx`.
En este archivo encontraremos los mismo datos del enunciado.
Importamos las hojas `CACs` y `Depositos` del archivo `Soporte Caso 5.xlsx`.
Estas hojas son importadas como objetos `DataFrame` de `pandas`.

In [2]:
cacs = pd.read_excel('Soporte Caso 5.xlsx', sheet_name='CACs')
depositos = pd.read_excel('Soporte Caso 5.xlsx', sheet_name='Depositos')

In [3]:
cacs

,Municipio,Latitud,Longitud,Produccion
0,"Andes, Antioquia",5.62412,-75.95589,35.37
1,"Medellín, Antioquia",6.26868,-75.59639,10.63
2,"Dabeiba, Antioquia",6.95267,-76.29085,27.28
3,"Salgar, Antioquia",5.96643,-75.97188,24.05
4,"San Pablo de Borbur, Boyacá",5.67784,-74.10383,1.59
5,"Labranzagrande, Boyacá",5.53000,-72.59873,1.20
6,"Miraflores, Boyacá",5.15175,-73.17282,1.11
7,"Moniquirá, Boyacá",5.86963,-73.54944,1.28
8,"Manizales, Caldas",5.07410,-75.50288,24.76
9,"Anserma, Caldas",5.20035,-75.75022,8.23


In [4]:
depositos

,Municipio,Latitud,Longitud,Capacidad,CostoFijo
0,"Medellín, Antioquia",6.26868,-75.59639,104,199194.44
1,"La Dorada, Caldas",5.53144,-74.72005,38,111741.67
2,"Aguadas, Caldas",5.57937,-75.45557,38,111741.67
3,"Salamina, Caldas",5.34395,-75.40658,35,107766.67
4,"Popayán, Cauca",2.44270,-76.57841,23,91863.89
5,"Valledupar, Cesar",10.46477,-73.25915,11,75963.89
6,"Bogotá, Cundinamarca",4.64830,-74.10781,10,74638.89
7,"Santana, Huila",3.58706,-74.70524,42,117041.67
8,"Neiva, Huila",3.03602,-75.29684,48,124991.67
9,"Santa Marta, Magdalena",11.23153,-74.18240,4,66688.89


### Procesamiento de archivos de soporte

En este paso, se crean los **Conjuntos** y **Parámetros**.
Es necesario dejar todo expresado en términos de listas y diccionarios para facilitar la implementación del modelo en PuLP. Adicionalmente, debemos procesar las coordenadas de longitud y latitud para obtener las distancias entre CACs y Depósitos.

In [5]:
I = depositos.Municipio.to_list()
J = cacs.Municipio.to_list()

print(I)
print(J)

['Medellín, Antioquia', 'La Dorada, Caldas ', 'Aguadas, Caldas', 'Salamina, Caldas', 'Popayán, Cauca', 'Valledupar, Cesar', 'Bogotá, Cundinamarca', 'Santana, Huila', 'Neiva, Huila', 'Santa Marta, Magdalena', 'Cúcuta, Nor. de Santander', 'Pasto, Nariño', 'Génova, Quindío', 'Calarcá, Quindío', 'Filandia, Quindío', 'Pereira, Risaralda', 'Bucaramanga, Santander', 'Barbosa, Santander', 'Cali, Valle del Cauca']
['Andes, Antioquia', 'Medellín, Antioquia', 'Dabeiba, Antioquia', 'Salgar, Antioquia', 'San Pablo de Borbur, Boyacá', 'Labranzagrande, Boyacá', 'Miraflores, Boyacá', 'Moniquirá, Boyacá', 'Manizales, Caldas', 'Anserma, Caldas', 'Pensilvania, Caldas', 'Riosucio, Caldas', 'Aguadas, Caldas', 'Morales, Cauca', 'El Tambo, Cauca', 'Bolívar, Cauca', 'Aguachica, Cesar', 'San Diego, Cesar', 'Caparrapí, Cundinamarca', 'Viotá, Cundinamarca', 'Sasaima, Cundinamarca', 'Neiva, Huila', 'Pitalito, Huila', 'Gigante, Huila', 'Santa Marta, Magdalena', 'La Unión, Nariño', 'Pasto, Nariño', 'Samaniego, Nari

In [6]:
capacidad = {row["Municipio"]: row["Capacidad"] for _, row in depositos.iterrows()}
costo_fijo = {row["Municipio"]: row["CostoFijo"] for _, row in depositos.iterrows()}
depositos_lat_lon = {
    row["Municipio"]: (row["Latitud"], row["Longitud"])
    for _, row in depositos.iterrows()
}

produccion = {row["Municipio"]: row["Produccion"] for _, row in cacs.iterrows()}
cacs_lat_lon = {
    row["Municipio"]: (row["Latitud"], row["Longitud"]) for _, row in cacs.iterrows()
}

q = 90  # Pesos anualizados por cada mil toneladas de café por kilómetro
r = 125  # Kilómetros

distancia = {
    (i, j): distance(depositos_lat_lon[i], cacs_lat_lon[j]).kilometers
    for i in I
    for j in J
}

In [7]:
distancia

{('Medellín, Antioquia', 'Andes, Antioquia'): 81.6407744153149,
 ('Medellín, Antioquia', 'Medellín, Antioquia'): 0.0,
 ('Medellín, Antioquia', 'Dabeiba, Antioquia'): 107.79278431038877,
 ('Medellín, Antioquia', 'Salgar, Antioquia'): 53.335670657757895,
 ('Medellín, Antioquia', 'San Pablo de Borbur, Boyacá'): 177.7022202999153,
 ('Medellín, Antioquia', 'Labranzagrande, Boyacá'): 341.843300383342,
 ('Medellín, Antioquia', 'Miraflores, Boyacá'): 295.50792681396376,
 ('Medellín, Antioquia', 'Moniquirá, Boyacá'): 230.85325502996707,
 ('Medellín, Antioquia', 'Manizales, Caldas'): 132.50832910815635,
 ('Medellín, Antioquia', 'Anserma, Caldas'): 119.36408078562327,
 ('Medellín, Antioquia', 'Pensilvania, Caldas'): 106.38936050657013,
 ('Medellín, Antioquia', 'Riosucio, Caldas'): 91.7929643629694,
 ('Medellín, Antioquia', 'Aguadas, Caldas'): 77.80655349448206,
 ('Medellín, Antioquia', 'Morales, Cauca'): 399.20320142759863,
 ('Medellín, Antioquia', 'El Tambo, Cauca'): 442.9927758625815,
 ('Medell

**Pregunta 1 (10 puntos)**

* Crea el parámetro de costo de transporte $c_{ij}$ en un diccionario llamado `costo_transporte`
* Las **llaves** de este diccionario deben ser los pares $(i,j)$, es decir, (depósitos, CACs)
* Los **valores** de este diccionario deben ser los costos de transporte definidos en la formulación

In [8]:
# your code here
costo_transporte = {
    (i, j): q * produccion[j] * distancia[(i, j)]
    for i in I
    for j in J
}

costo_transporte


{('Medellín, Antioquia', 'Andes, Antioquia'): 259887.0771962719,
 ('Medellín, Antioquia', 'Medellín, Antioquia'): 0.0,
 ('Medellín, Antioquia', 'Dabeiba, Antioquia'): 264652.8440388665,
 ('Medellín, Antioquia', 'Salgar, Antioquia'): 115445.05913871697,
 ('Medellín, Antioquia', 'San Pablo de Borbur, Boyacá'): 25429.18772491788,
 ('Medellín, Antioquia', 'Labranzagrande, Boyacá'): 36919.07644140094,
 ('Medellín, Antioquia', 'Miraflores, Boyacá'): 29521.241888714983,
 ('Medellín, Antioquia', 'Moniquirá, Boyacá'): 26594.294979452206,
 ('Medellín, Antioquia', 'Manizales, Caldas'): 295281.5605846156,
 ('Medellín, Antioquia', 'Anserma, Caldas'): 88412.97463791116,
 ('Medellín, Antioquia', 'Pensilvania, Caldas'): 63195.28014090266,
 ('Medellín, Antioquia', 'Riosucio, Caldas'): 54772.86183538385,
 ('Medellín, Antioquia', 'Aguadas, Caldas'): 47197.45534975282,
 ('Medellín, Antioquia', 'Morales, Cauca'): 509822.40854318615,
 ('Medellín, Antioquia', 'El Tambo, Cauca'): 528268.8852161284,
 ('Medellí

In [5]:
# Esta celda esta reservada para uso del equipo docente

In [6]:
# Esta celda esta reservada para uso del equipo docente

**Celda de Prueba (0 puntos)**

Es una buena práctica imprimir algunos objetos que contienen los parámetros en la consola luego de crearlos. De esta forma puedes corregir errores y familiarizarte con las estructuras de datos que se van a utilizar. Puedes hacer estas pruebas en la celda a continuación.

* **Esta celda no es calificable**

In [7]:
# your code here


## Modelado - Minimización de costos ($z_1$)
---

### Declaración del modelo

In [9]:
problema = lp.LpProblem(sense=lp.LpMinimize)

### Variables de Decisión

>* $x_{ij}=\begin{cases}1, & \text{si el CAC } j \in J \text{ es atendido por el depósito } i \in I \\0, & \text{de lo contrario} \end{cases}$
>* $y_{i}=\begin{cases} 1, & \text{si se decide operar el depósito } i \in I  \\ 0, & \text{de lo contrario} \end{cases}$

In [10]:
x = lp.LpVariable.dicts("atender", [(i, j) for i in I for j in J], lowBound = 0, cat=lp.LpBinary)
y = lp.LpVariable.dicts("operar", I, lowBound = 0, cat=lp.LpBinary)

### Función Objetivo
Minimizar los costos totales de operación y transporte
>`# Para desarrollo del estudiante`

**Pregunta 2 (10 puntos)**
* Crea la función objetivo y agrégala al modelo `problema`

> **Ejemplo**:
>> $ \sum_{i \in I}c_i x_i$
es equivalente a `lp.lpSum(c[i]*x[i] for i in I)`

In [11]:
# your code here
# Función objetivo: minimizar el costo total de operación y transporte

# La función objetivo suma dos componentes:
# 1. El costo fijo de operar cada depósito seleccionado
# 2. El costo de transporte de atender cada CAC desde cada depósito asignado
problema += (
    lp.lpSum(costo_fijo[i] * y[i] for i in I) +  # Costo fijo de operación de los depósitos
    lp.lpSum(costo_transporte[i, j] * x[i, j] for i in I for j in J)  # Costo de transporte de los CACs
), "Costo_Total"

In [11]:
# Esta celda esta reservada para uso del equipo docente

In [12]:
# Esta celda esta reservada para uso del equipo docente

### Restricciones

____
**Ejemplo**
> La siguiente restricción: $\sum_{i \in I} a_{ij} x_{ij} \geq 1, \; \forall j \in J$ es equivalente a:
>    * `for j in J:`
>        * `model += lp.lpSum(a[i,j]*x[i,j] for i in I) >= 1, 'R1_'+str(j)`
    
**Advertencia**: En `pulp` no es recomendable sobreescribir restricciones, entonces, si ya creaste una restricción y quieres crearla de nuevo para corregir algo, asegúrate de volver a crear el modelo `problema` desde el principio. (Nosotros haremos esto antes de calificar, no te preocupes)

**Pregunta 3 (10 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R1_'+str(<indice_del_para_todo>)` y añádela al modelo:
>1. Cada CAC debe ser atendido por un único depósito
>>`# Para desarrollo del estudiante`

In [12]:
# your code here
# Restricción: cada CAC debe ser atendido por un único depósito
for j in J:
    problema += lp.lpSum(x[i, j] for i in I) == 1, 'R1_' + str(j)


In [14]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 4 (5 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R2_'+str(<indice_del_para_todo>)` y añádela al modelo:
>2. No se debe superar la capacidad de los depósitos y sólo se puede atender CACs desde un depósito si se decide operar el mismo.
>>$\sum_{j \in J}d_{j}x_{ij} \leq k_{i}y_{i}, \; \forall i \in I$

In [13]:
# your code here
# Restricción: no superar la capacidad de los depósitos y solo operar si se decide abrir el depósito
for i in I:
    problema += lp.lpSum(produccion[j] * x[i, j] for j in J) <= capacidad[i] * y[i], 'R2_' + str(i)


In [16]:
# Esta celda esta reservada para uso del equipo docente

In [17]:
# Esta celda esta reservada para uso del equipo docente

### Invocar el optimizador

In [14]:
print(lp.LpStatus[problema.solve()])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/edwin/.local/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/15da84e2c5ed4c90a2a2c2bb1a1cef0d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/15da84e2c5ed4c90a2a2c2bb1a1cef0d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 79 COLUMNS
At line 5371 RHS
At line 5446 BOUNDS
At line 6511 ENDATA
Problem MODEL has 74 rows, 1064 columns and 2109 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.92439e+06 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 74 rows, 874 columns (874 integer (874 of which binary)) and 1729 elements
Cbc0038I Initial state - 29 integers unsatisfied sum - 7.38792
Cbc0038I Pass   1: suminf.    4.28652 (20) obj. 4.13822e+06 iterations 50
Cbc0038I Pass 

## Reporte de resultados - Minimización de costos ($z_1$)
---

**Función objetivo $z_1$**

In [15]:
z1 = lp.value(problema.objective)
min_costo = z1
print(f"Costo Total: ${z1: .2f}")
print(f"Costo Total Relativo al Mínimo Costo: {z1/min_costo*100: .2f}%")

Costo Total: $ 4318336.74
Costo Total Relativo al Mínimo Costo:  100.00%


In [16]:
z1 = lp.value(problema.objective)
min_costo = z1
print(f"Costo Total: ${z1: .2f}")
print(f"Costo Total Relativo al Mínimo Costo: {z1/min_costo*100: .2f}%")

Costo Total: $ 4318336.74
Costo Total Relativo al Mínimo Costo:  100.00%


**Función objetivo $z_2$**

**Pregunta 5 (5 puntos)**

* Guarda en una variable `z2` el valor de la satisfacción total dado por la expresión:
> $z_2 = \sum_{j \in J} d_{j} \sum_{\{i \in I | h_{ij} \leq r\}} x_{ij}$

**Recuerda que** en PuLP puedes usar la función `lp.value(<expresion>)` para evaluar una expresión, reemplazando los valores de las variables por aquellos de la solución óptima. Esta función sólo debe ser llamada luego de usar `<modelo>.solve()` y haber obtenido una solución óptima.

In [17]:
# your code here
z2 = lp.value(lp.lpSum(
    produccion[j] * x[i, j]
    for j in J
    for i in I
    if distancia[(i, j)] <= r
) )
z2

373.75000000000006

Satisfacción Total: 373.75
Satisfacción Total Relativa al Total de Producción: 88.94%


In [22]:
# Esta celda esta reservada para uso del equipo docente

**Depósitos en operación**

In [18]:
print("Se decidió operar", sum(y[i].value() for i in I), "depósitos")

Se decidió operar 15.0 depósitos


**Asignación de CACs a Depósitos**

In [19]:
matrix = []
for j in J:
    row = []
    for i in I:
        if y[i].value() == 1:
            if x[i,j].value() == 1:
                row.append('X')
            elif x[i,j].value() == 0:
                row.append('-')
            else:
                row.append('Error')
    matrix.append(row)
    
df = pd.DataFrame(matrix, index=J, columns=[i for i in I if y[i].value() == 1])
df.head(10)

,"Medellín, Antioquia","La Dorada, Caldas","Aguadas, Caldas","Salamina, Caldas","Popayán, Cauca","Valledupar, Cesar","Santana, Huila","Neiva, Huila","Cúcuta, Nor. de Santander","Pasto, Nariño","Génova, Quindío","Filandia, Quindío","Bucaramanga, Santander","Barbosa, Santander","Cali, Valle del Cauca"
"Andes, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Medellín, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Dabeiba, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Salgar, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"San Pablo de Borbur, Boyacá",-,X,-,-,-,-,-,-,-,-,-,-,-,-,-
"Labranzagrande, Boyacá",-,-,-,-,-,-,-,-,-,-,-,-,X,-,-
"Miraflores, Boyacá",-,-,-,-,-,-,-,-,-,-,-,-,-,X,-
"Moniquirá, Boyacá",-,-,-,-,-,-,-,-,-,-,-,-,-,X,-
"Manizales, Caldas",-,-,-,X,-,-,-,-,-,-,-,-,-,-,-
"Anserma, Caldas",-,-,X,-,-,-,-,-,-,-,-,-,-,-,-


### Visualizaciones
---

**Mapa de la asignación**

In [20]:
# Para los mapas
import folium
# Para los marcadores de los mapas
from folium.plugins import BeautifyIcon

m = folium.Map(location=[6.2, -74.5], tiles="OpenStreetMap", zoom_start=6)

for j, lat_lon in cacs_lat_lon.items():
    folium.Marker(
        location=lat_lon,
        tooltip=j,
        icon=BeautifyIcon(
            icon="circle",
            inner_icon_style="color:blue;font-size:7px;opacity:0.9;position: relative;top:-0.5px;",
            background_color="transparent",
            border_color="transparent",
        ),
    ).add_to(m)
for i, lat_lon in depositos_lat_lon.items():
    if y[i].value() > 0:
        folium.Marker(
            location=lat_lon,
            tooltip=i,
            icon=BeautifyIcon(
                icon="caret-up",
                inner_icon_style="color:red;font-size:20px;opacity:0.9;position: relative;top:-4.5px;",
                background_color="transparent",
                border_color="transparent",
            ),
        ).add_to(m)

red = [(i, j) for i in I for j in J if x[i, j].value() > 0]
for i, j in red:
    folium.PolyLine(
        [depositos_lat_lon[i], cacs_lat_lon[j]], color="black", weight=1, opacity=1
    ).add_to(m)

m

## Modelado - Maximización de satisfacción ($z_2$)
---
A continuación queremos explorar el cambio en las funciones objetivo $z_1$ y $z_2$ cuando se prioriza $z_2$. Las restricciones y variables del problema permanecen igual, pero la solución cambiará.

### Declaración del modelo

In [21]:
problema = lp.LpProblem(sense=lp.LpMaximize)

### Variables de Decisión

>* $x_{ij}=\begin{cases}1, & \text{si el CAC } j \in J \text{ es atendido por el depósito } i \in I \\0, & \text{de lo contrario} \end{cases}$
>* $y_{i}=\begin{cases} 1, & \text{si se decide operar el depósito } i \in I  \\ 0, & \text{de lo contrario} \end{cases}$

In [22]:
x = lp.LpVariable.dicts('atender', [(i,j) for i in I for j in J], cat=lp.LpBinary)
y = lp.LpVariable.dicts('operar', I, cat=lp.LpBinary)

### Función Objetivo
Maximizar satisfacción de los CACs
>$\max z_2 = \sum_{j \in J} d_{j} \sum_{\{i \in I | h_{ij} \leq r\}} x_{ij}.$

Esta expresión es equivalente a:
>$\max z_2 = \sum_{j \in J} \sum_{\{i \in I | h_{ij} \leq r\}}d_{j} x_{ij}.$

**Pregunta 6 (5 puntos)**
* Crea la función objetivo y agrégala al modelo `problema`

> **Ejemplo**:
>> $ \sum_{i \in I}c_i x_i$
es equivalente a `lp.lpSum(c[i]*x[i] for i in I)`

In [25]:
# your code here
problema += lp.lpSum(
    produccion[j] * x[i, j]
    for j in J
    for i in I
    if distancia[(i, j)] <= r
) , "Satisfaccion_Total"

In [29]:
# Esta celda esta reservada para uso del equipo docente

In [30]:
# Esta celda esta reservada para uso del equipo docente

### Restricciones
____

**Pregunta 7 (10 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R1_'+str(<indice_del_para_todo>)` y añádela al modelo:
>1. Cada CAC debe ser atendido por un único depósito
>>`# Para desarrollo del estudiante`

In [26]:
# your code here
for j in J:
    problema += lp.lpSum(x[i, j] for i in I) == 1, 'R1_' + str(j)

In [32]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 8 (5 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R2_'+str(<indice_del_para_todo>)` y añádela al modelo:
>2. No se debe superar la capacidad de los depósitos y sólo se puede atender CACs desde un depósito si se decide operar el mismo.
>>$\sum_{j \in J}d_{j}x_{ij} \leq k_{i}y_{i}, \; \forall i \in I$

In [27]:
# your code here
for i in I:
    problema += lp.lpSum(produccion[j] * x[i, j] for j in J) <= capacidad[i] * y[i], 'R2_' + str(i)

In [34]:
# Esta celda esta reservada para uso del equipo docente

In [35]:
# Esta celda esta reservada para uso del equipo docente

### Invocar el optimizador

In [28]:
print(lp.LpStatus[problema.solve()])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/edwin/.local/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/d8a4b82c7f554cb59595e46d84e7c84d-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/d8a4b82c7f554cb59595e46d84e7c84d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 79 COLUMNS
At line 4495 RHS
At line 4570 BOUNDS
At line 5635 ENDATA
Problem MODEL has 74 rows, 1064 columns and 2109 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 406.42 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0005I 55 SOS with 855 members
Cgl0004I processed model has 74 rows, 855 columns (855 integer (855 of which binary)) and 1710 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 18 integers unsatisfied sum - 4.32306
Cbc003

## Reporte de resultados - Maximización de satisfacción ($z_2$)
---

**Función objetivo $z_1$**

**Pregunta 9 (5 puntos)**

* Guarda en una variable `z1` el valor del costo total de operación y transporte:
>`# Para desarrollo del estudiante`

**Recuerda que** en PuLP puedes usar la función `lp.value(<expresion>)` para evaluar una expresión, reemplazando los valores de las variables por aquellos de la solución óptima. Esta función sólo debe ser llamada luego de usar `<modelo>.solve()` y haber obtenido una solución óptima.

In [30]:
# your code here
z1 = lp.value(
    lp.lpSum(costo_fijo[i] * y[i] for i in I) +
    lp.lpSum(costo_transporte[i, j] * x[i, j] for i in I for j in J)
    )
z1

4837569.383828101

In [31]:
print(f"Costo Total: ${z1: .2f}")
print(f"Costo Total Relativo al Mínimo Costo: {z1 / min_costo * 100: .2f}%")

Costo Total: $ 4837569.38
Costo Total Relativo al Mínimo Costo:  112.02%


In [39]:
# Esta celda esta reservada para uso del equipo docente

**Función objetivo $z_2$**

In [32]:
z2 = lp.value(problema.objective)
print(f"Satisfacción Total: {z2: .2f}")
print(
    f"Satisfacción Total Relativa al Total de Producción: {z2 / sum(produccion.values()) * 100: .2f}%"
)

Satisfacción Total:  398.33
Satisfacción Total Relativa al Total de Producción:  94.79%


**Depósitos en operación**

In [33]:
print("Se decidió operar", sum([y[i].value() for i in I]), "depósitos")

Se decidió operar 19.0 depósitos


**Asignación de CACs a Depósitos**

In [34]:
matrix = []
for j in J:
    row = []
    for i in I:
        if y[i].value() == 1:
            if x[i, j].value() == 1:
                row.append("X")
            elif x[i, j].value() == 0:
                row.append("-")
            else:
                row.append("Error")
    matrix.append(row)

df = pd.DataFrame(matrix, index=J, columns=[i for i in I if y[i].value() == 1])
df.head(10)

,"Medellín, Antioquia","La Dorada, Caldas","Aguadas, Caldas","Salamina, Caldas","Popayán, Cauca","Valledupar, Cesar","Bogotá, Cundinamarca","Santana, Huila","Neiva, Huila","Santa Marta, Magdalena","Cúcuta, Nor. de Santander","Pasto, Nariño","Génova, Quindío","Calarcá, Quindío","Filandia, Quindío","Pereira, Risaralda","Bucaramanga, Santander","Barbosa, Santander","Cali, Valle del Cauca"
"Andes, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Medellín, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Dabeiba, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Salgar, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"San Pablo de Borbur, Boyacá",-,-,-,-,-,-,X,-,-,-,-,-,-,-,-,-,-,-,-
"Labranzagrande, Boyacá",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,X,-
"Miraflores, Boyacá",-,-,-,-,-,-,X,-,-,-,-,-,-,-,-,-,-,-,-
"Moniquirá, Boyacá",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,X,-
"Manizales, Caldas",-,-,-,X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Anserma, Caldas",-,X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


### Visualizaciones
---

**Mapa de la asignación**

In [35]:
m = folium.Map(location=[6.2, -74.5], tiles="OpenStreetMap", zoom_start=6)

for j, lat_lon in cacs_lat_lon.items():
    folium.Marker(
        location=lat_lon,
        tooltip=j,
        icon=BeautifyIcon(
            icon="circle",
            inner_icon_style="color:blue;font-size:7px;opacity:0.9;position: relative;top:-0.5px;",
            background_color="transparent",
            border_color="transparent",
        ),
    ).add_to(m)
for i, lat_lon in depositos_lat_lon.items():
    if y[i].value() > 0:
        folium.Marker(
            location=lat_lon,
            tooltip=i,
            icon=BeautifyIcon(
                icon="caret-up",
                inner_icon_style="color:red;font-size:20px;opacity:0.9;position: relative;top:-4.5px;",
                background_color="transparent",
                border_color="transparent",
            ),
        ).add_to(m)

red = [(i, j) for i in I for j in J if x[i, j].value() > 0]
for i, j in red:
    folium.PolyLine(
        [depositos_lat_lon[i], cacs_lat_lon[j]], color="black", weight=1, opacity=1
    ).add_to(m)

m

## Modelado - Maximización de satisfacción ($z_2$) con restricción de costos ($z_1$)
---
Por último, queremos encontrar un solución intermedia entre la que minimiza los costos y la que maximiza la satisfacción. Hay varias maneras de hacer esto. La que vamos a implementar es colocar una restricción sobre los costos $z_1$ que esté entre los valores obtenidos en los dos casos anteriores.

Recordemos que al minimizar los costos, se obtuvo un costo total de 4,318,336.74. Este costo será nuestro punto de referencia. No podemos obtener un costo menor a este. Por otro lado, al maximizar la satisfacción, obtuvimos un costo de 4,837,569.38. Así que en el peor de los casos, el costo es aproximadamente 12.02% mayor al primer caso. Entonces, para obtener una solución intermedia, debemos escoger un umbral entre estos dos valores para crear una restricción sobre los costos. Una posibilidad es restringir que el costo total $z_1$ sea a lo sumo 2% mayor que el mejor costo mientras se maximiza la satisfacción $z_2$. 

### Declaración del modelo

In [36]:
problema = lp.LpProblem(sense=lp.LpMaximize)

### Variables de Decisión

>* $x_{ij}=\begin{cases}1, & \text{si el CAC } j \in J \text{ es atendido por el depósito } i \in I \\0, & \text{de lo contrario} \end{cases}$
>* $y_{i}=\begin{cases} 1, & \text{Si se decide operar el depósito } i \in I  \\ 0, & \text{de lo contrario} \end{cases}$

In [37]:
x = lp.LpVariable.dicts("atender", [(i, j) for i in I for j in J], cat=lp.LpBinary)
y = lp.LpVariable.dicts("operar", I, cat=lp.LpBinary)

### Función Objetivo
Maximizar satisfacción de los CACs
>$\max z_2 = \sum_{j \in J} d_{j} \sum_{\{i \in I | h_{ij} \leq r\}} x_{ij}$

**Pregunta 10 (5 puntos)**
* Crea la función objetivo y agrégala al modelo `problema`

> **Ejemplo**:
>> $ \sum_{i \in I}c_i x_i$
es equivalente a `lp.lpSum(c[i]*x[i] for i in I)`

In [38]:
# your code here
problema += lp.lpSum(
    produccion[j] * x[i, j]
    for j in J
    for i in I
    if distancia[(i, j)] <= r
) , "Satisfaccion_Total"

In [47]:
# Esta celda esta reservada para uso del equipo docente

In [48]:
# Esta celda esta reservada para uso del equipo docente

### Restricciones
____

**Pregunta 11 (10 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R1_'+str(<indice_del_para_todo>)` y añádela al modelo:
>1. Cada CAC debe ser atendido por un único depósito
>>`# Para desarrollo del estudiante`

In [39]:
# your code here
for j in J:
    problema += lp.lpSum(x[i, j] for i in I) == 1, 'R1_' + str(j)

In [50]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 12 (5 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R2_'+str(<indice_del_para_todo>)` y añádela al modelo:
>2. No se debe superar la capacidad de los depósitos y sólo se puede atender CACs desde un depósito si se decide operar el mismo.
>>$\sum_{j \in J}d_{j}x_{ij} \leq k_{i}y_{i}, \; \forall i \in I$

In [40]:
# your code here
for i in I:
    problema += lp.lpSum(produccion[j] * x[i, j] for j in J) <= capacidad[i] * y[i], 'R2_' + str(i)

In [52]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 13 (10 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R3'` y añádela al modelo:
>El costo total no debe superar en más de un 2% al mejor costo obtenido.
>>`# Para desarrollo del estudiante` 

**Nota:** Utiliza el valor `z1_` a continuación como el mejor costo obtenido. Inclúyelo en la restricción según sea conveniente.

In [42]:
z1_ = 4318336.74

In [43]:
# your code here
problema += (
    lp.lpSum(costo_fijo[i] * y[i] for i in I) +
    lp.lpSum(costo_transporte[i, j] * x[i, j] for i in I for j in J)
    <= z1_ * 1.02
), 'R3'

In [55]:
# Esta celda esta reservada para uso del equipo docente

In [56]:
# Esta celda esta reservada para uso del equipo docente

### Invocar el optimizador

In [44]:
print(lp.LpStatus[problema.solve()])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/edwin/.local/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/8574db50869d49909850cfe1333600a3-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/8574db50869d49909850cfe1333600a3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 80 COLUMNS
At line 5550 RHS
At line 5626 BOUNDS
At line 6691 ENDATA
Problem MODEL has 75 rows, 1064 columns and 3163 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 406.42 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 75 rows, 874 columns (874 integer (874 of which binary)) and 2595 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 27 integers unsatisfied sum - 6.66021
Cbc0038I Pass   1: suminf.    4.44054 (

## Reporte de resultados - Maximización de satisfacción ($z_2$) con restricción de costos ($z_1$)
---

**Función objetivo $z_1$**

**Pregunta 14 (5 puntos)**

* Guarda en una variable `z1` el valor del costo total de operación y transporte:
>`# Para desarrollo del estudiante`

**Recuerda que** en PuLP puedes usar la función `lp.value(<expresion>)` para evaluar una expresión, reemplazando los valores de las variables por aquellos de la solución óptima. Esta función sólo debe ser llamada luego de usar `<modelo>.solve()` y haber obtenido una solución óptima.

In [45]:
# your code here
z1 = float(format(lp.value(lp.lpSum(costo_fijo[i] * y[i] for i in I)
+ lp.lpSum(costo_transporte[i,j]* x[i, j] for i in I for j in J))))
z1

4395103.84036442

In [46]:
print(f"Costo Total: ${z1: .2f}")
print(f"Costo Total Relativo al Mínimo Costo: {z1 / min_costo * 100: .2f}%")

Costo Total: $ 4395103.84
Costo Total Relativo al Mínimo Costo:  101.78%


In [60]:
# Esta celda esta reservada para uso del equipo docente

**Función objetivo $z_2$**

In [47]:
z2 = lp.value(problema.objective)
print(f"Satisfacción Total: {z2: .2f}")
print(f"Satisfacción Total Relativa al Total de Producción: {z2 / sum(produccion.values()) * 100: .2f}%")

Satisfacción Total:  396.67
Satisfacción Total Relativa al Total de Producción:  94.39%


**Depósitos en operación**

In [48]:
print("Se decidió operar", sum(y[i].value() for i in I), "depósitos")

Se decidió operar 17.0 depósitos


**Asignación de CACs a Depósitos**

In [49]:
matrix = []
for j in J:
    row = []
    for i in I:
        if y[i].value() == 1:
            if x[i, j].value() == 1:
                row.append("X")
            elif x[i, j].value() == 0:
                row.append("-")
            else:
                row.append("Error")
    matrix.append(row)

df = pd.DataFrame(matrix, index=J, columns=[i for i in I if y[i].value() == 1])
df.head(10)

,"Medellín, Antioquia","La Dorada, Caldas","Aguadas, Caldas","Salamina, Caldas","Popayán, Cauca","Valledupar, Cesar","Santana, Huila","Neiva, Huila","Santa Marta, Magdalena","Cúcuta, Nor. de Santander","Pasto, Nariño","Génova, Quindío","Calarcá, Quindío","Filandia, Quindío","Bucaramanga, Santander","Barbosa, Santander","Cali, Valle del Cauca"
"Andes, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Medellín, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Dabeiba, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Salgar, Antioquia",X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"San Pablo de Borbur, Boyacá",-,X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Labranzagrande, Boyacá",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,X,-
"Miraflores, Boyacá",-,X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
"Moniquirá, Boyacá",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,X,-
"Manizales, Caldas",-,-,-,X,-,-,-,-,-,-,-,-,-,-,-,-,-
"Anserma, Caldas",-,X,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


### Visualizaciones
---

**Mapa de la asignación**

In [52]:
m = folium.Map(location=[6.2, -74.5], tiles="OpenStreetMap", zoom_start=6)

for j, lat_lon in cacs_lat_lon.items():
    folium.Marker(
        location=lat_lon,
        tooltip=j,
        icon=BeautifyIcon(
            icon="circle",
            inner_icon_style="color:blue;font-size:7px;opacity:0.9;position: relative;top:-0.5px;",
            background_color="transparent",
            border_color="transparent",
        ),
    ).add_to(m)

for i, lat_lon in depositos_lat_lon.items():
    if y[i].value() > 0:
        folium.Marker(
            location=lat_lon,
            tooltip=i,
            icon=BeautifyIcon(
                icon="caret-up",
                inner_icon_style="color:red;font-size:20px;opacity:0.9;position: relative;top:-4.5px;",
                background_color="transparent",
                border_color="transparent",
            ),
        ).add_to(m)

red = [(i, j) for i in I for j in J if x[i, j].value() > 0]
for i, j in red:
    folium.PolyLine(
        [depositos_lat_lon[i], cacs_lat_lon[j]], color="black", weight=1, opacity=1
    ).add_to(m)
m

### Reflexión
---
¿De qué forma podrías obtener soluciones intermedias adicionales? ¿Podrías presentarlas gráficamente como una frontera de Pareto? ¿Si tuvieras que recomendar alguna solución, con qué criterio la escogerías?